# Import des librairies 

In [5]:
import pandas as pd 
from IPython.display import clear_output
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import csv
import numpy as np
import re
from tqdm import tqdm
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings('ignore')

In [6]:
data_path="Data/"#Chemin du dossier pour enregistrer le fichier csv

In [7]:
def check_nan(df):
    for i in df.columns.tolist():
        print("Valeurs nan dans "+str(i)+" : "+str(df[i].isna().sum()))
        
def check_unique(df):
    for i in df.columns.tolist():
        print("Valeurs uniques dans "+str(i)+" : "+str(df[i].nunique()))

# Scrapping des ID et titres en français des films Allociné

In [36]:
%%time
# Initialisation des variables pour stocker les données
data = []
page_number=1
# Boucle pour récupérer les données de chaque page
while len(data)<20000:
    print("Page :",page_number)
    print("Len data :",len(data))
    url = f"https://www.allocine.fr/films/?page={page_number}"
    req = Request(
        url=url, 
        headers={'User-Agent': 'Mozilla/5.0'}
    )
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    movies = soup.find_all("a",{"class" : "meta-title-link"})

    # Récupération des données de chaque film
    if len(movies)==0 : 
        break
    for movie in movies:
        film_data = {
            'titre':movie.text,
            'id': re.sub(r"\D", "",movie.get("href")),
        }
        data.append(film_data)
    page_number += 1
    clear_output()
df=pd.DataFrame(data)
df.head()

CPU times: total: 2min 38s
Wall time: 9min 47s


,titre,id
0,Avatar : la voie de l'eau,178014
1,Les Banshees d'Inisherin,281293
2,Tempête,289305
3,M3GAN,266320
4,Le Tourbillon de la vie,288544


# Sauvegardes de données

In [45]:
df["titre"]= df["titre"].str.lower()
df= df.drop_duplicates()
df.to_csv(data_path+"allocine_titre_id.csv",index=False)

# Observations des données 

In [8]:
df = pd.read_csv(data_path+"allocine_titre_id.csv")
check_nan(df)
print()
check_unique(df)
print()
print("Shape :",df.shape)

Valeurs nan dans titre : 0
Valeurs nan dans id : 0

Valeurs uniques dans titre : 19141
Valeurs uniques dans id : 19982

Shape : (20010, 2)
